In [ ]:
%matplotlib inline


# Immunohistochemical staining colors separation


Color deconvolution consists of the separation of features by their colors.

In this example we separate the immunohistochemical (IHC) staining from the
hematoxylin counterstaining. The separation is achieved with the method
described in [1]_, known as "color deconvolution".

The IHC staining expression of the FHL2 protein is here revealed with
Diaminobenzidine (DAB) which gives a brown color.


.. [1] A. C. Ruifrok and D. A. Johnston, "Quantification of histochemical
       staining by color deconvolution.," Analytical and quantitative
       cytology and histology / the International Academy of Cytology [and]
       American Society of Cytology, vol. 23, no. 4, pp. 291-9, Aug. 2001.


## HED颜色空间  
苏木精 — 伊红染色法 ( hematoxylin-eosin staining ) ，简称HE染色法 ，石蜡切片技术里常用的染色法之一 。苏木精染液为碱性 ，主要使细胞核内的染色质与胞质内的核酸着紫蓝色 ；伊红为酸性染料 ，主要使细胞质和细胞外基质中的成分着红色 。HE染色法是组织学、胚胎学、病理学教学与科研中最基本、使用最广泛的技术方法。


In [1]:
import matplotlib.pyplot as plt

from skimage import data, viewer
from skimage.color import rgb2hed
from matplotlib.colors import LinearSegmentedColormap

# Create an artificial color close to the orginal one
cmap_hema = LinearSegmentedColormap.from_list('mycmap', ['white', 'navy'])
cmap_dab = LinearSegmentedColormap.from_list('mycmap', ['white',
                                                        'saddlebrown'])
cmap_eosin = LinearSegmentedColormap.from_list('mycmap', ['darkviolet',
                                                          'white'])

#读取文件
ihc_rgb = data.immunohistochemistry()
#颜色空间转换RGB->HED
ihc_hed = rgb2hed(ihc_rgb)

fig, axes = plt.subplots(2, 2, figsize=(7, 6), sharex=True, sharey=True,
                         subplot_kw={'adjustable': 'box-forced'})
#把二维数组展平
ax = axes.ravel()

ax[0].imshow(ihc_rgb)
ax[0].set_title("Original image")

#使用imshow的cmap参数指定颜色空间转换
ax[1].imshow(ihc_hed[:, :, 0], cmap=cmap_hema)
ax[1].set_title("Hematoxylin")  #hematoxylin苏木精，一种碱性染液
#苏木精-伊红染色法

ax[2].imshow(ihc_hed[:, :, 1], cmap=cmap_eosin)
ax[2].set_title("Eosin")

ax[3].imshow(ihc_hed[:, :, 2], cmap=cmap_dab)
ax[3].set_title("DAB")

#关闭坐标轴
for a in ax.ravel():
    a.axis('off')

fig.tight_layout()


Now we can easily manipulate the hematoxylin and DAB "channels":



In [2]:
import numpy as np

from skimage.exposure import rescale_intensity

# Rescale hematoxylin and DAB signals and give them a fluorescence look
#在HED颜色空间中有三个通道H，E，D，现在将H和D两个通道进行归一化操作
h = rescale_intensity(ihc_hed[:, :, 0], out_range=(0, 1))
d = rescale_intensity(ihc_hed[:, :, 2], out_range=(0, 1))
zdh = np.dstack((np.zeros_like(h), d, h))

fig = plt.figure()
axis = plt.subplot(1, 1, 1, sharex=ax[0], sharey=ax[0], adjustable='box-forced')
axis.imshow(zdh)
axis.set_title("Stain separated image (rescaled)")
axis.axis('off')
plt.show()